## `__new__`

Used to create a new instance of a class before it is initialized. The __new__ method is a static class method, not an instance method, and it's called when an object is created. It is responsible for returning a new instance of the class.

- It is the first step of instance creation, invoked before __init__.
- It takes the class as the first argument and usually forwards any extra arguments to the __init__ method.
- It's rarely used, but there are cases where it is necessary or beneficial to override __new__.
- It's commonly overridden in the creation of immutable types or metaclasses, or when subclassing immutable built-in types like str, int, tuple, etc.

In [ ]:
class MyClass(object):

    def __new__(cls, *args, **kwargs):
        # This is where you would put extra logic that needs to happen
        # before an object gets created.

        # Create the instance by calling the superclass's __new__ method.
        instance = super(MyClass, cls).__new__(cls)

        # You could also customize instance creation here by not calling super.
        # Example: object.__new__(cls, *args, **kwargs)

        return instance

    def __init__(self, *args, **kwargs):
        # This is where the object is actually initialized.
        print("The instance is created and now being initialized")
        # Initialize the instance as usual.


Example

## Examples

In [ ]:

#1. Creating a Singleton class:
 #  A Singleton is a design pattern that restricts the instantiation of a class to one "single" instance. Here's how you can use `__new__` to create a Singleton class:


class Singleton:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            print("Creating the Singleton instance")
            cls._instance = super(Singleton, cls).__new__(cls)
        return cls._instance

singleton1 = Singleton()
singleton2 = Singleton()

print(singleton1 is singleton2)  # Output: True


In [ ]:

#2. Extending an immutable type:
 #  Here's an example of extending Python's immutable `tuple` type by overriding `__new__`:

class VerboseTuple(tuple):
    def __new__(cls, iterable):
        print("Creating a VerboseTuple object with elements:", iterable)
        return super(VerboseTuple, cls).__new__(cls, iterable)

verbose_tuple = VerboseTuple((1, 2, 3))  # Output: Creating a VerboseTuple object with elements: (1, 2, 3)


In [ ]:

#3. Customizing instance creation:
 #  You can override `__new__` to customize the creation of instances, such as intercepting creation parameters or enforcing additional constraints:

class NonNegativeNumber:
    def __new__(cls, value):
        if value < 0:
            raise ValueError("Cannot instantiate with a negative number")
        instance = super(NonNegativeNumber, cls).__new__(cls)
        instance.value = value
        return instance

    def __init__(self, value):
        # Initialization logic here (already done in __new__ in this case)
        pass

try:
    non_negative = NonNegativeNumber(-10)  # This will raise a ValueError
except ValueError as e:
    print(e)  # Output: Cannot instantiate with a negative number



In each of these examples, `__new__` is used to control the creation of a new instance of the class. It's a powerful tool that allows for flexibility, but it should be used judiciously since it's a lower-level method and can lead to more complex and harder-to-understand code if not used properly.

## `__slots__`

The `__slots__` declaration in Python is used within a class definition to declare fixed attributes and properties for instances. Here are the main benefits of using `__slots__`:

1. **Memory Savings**: When you use `__slots__`, Python does not need to use a dictionary to store instance variables. Since dictionaries consume a significant amount of memory (due to their underlying hash table structure), replacing them with a static structure can lead to substantial memory savings, especially in programs that create many instances of a class.

2. **Faster Attribute Access**: Accessing attributes via `__slots__` is faster than accessing them from a dynamic `__dict__`. This is because slotted attributes are stored in a fixed-size array, not in a dictionary that requires hashing keys and resolving collisions.

3. **Preventing Arbitrary New Attributes**: Using `__slots__` can prevent the addition of new attributes to instances, which can be an aid in avoiding bugs due to typoed attribute names. It enforces a more strict class definition, which can help catch errors earlier in the development cycle.

4. **Explicit Attributes Declaration**: `__slots__` forces you to explicitly declare data members, which can make it clearer to new readers of the code what the intention of the class design was.

Why `__slots__` exists:

`__slots__` exists primarily to provide a memory optimization tool. Python objects can be quite memory-intensive, due to the dynamic nature of their attribute storage. Each instance has a dictionary to hold attribute values, which allows for the dynamic addition of attributes but at a memory cost. When you have thousands or millions of instances, that cost adds up.

In situations where you know that the set of attributes on instances of a class is going to be fixed, `__slots__` gives you the ability to avoid that overhead. This is particularly useful for classes that might be used as elements in data-intensive scientific computing or in situations where you might be dealing with a large in-memory graph of objects, such as a complex parser.

It's important to note that `__slots__` is not always the right choice. It should be used when the benefits outweigh the downsides, such as the loss of dynamic attribute assignment and the inability to support weak references (unless `'__weakref__'` is explicitly included in `__slots__`). Additionally, subclassing needs to be handled with care when `__slots__` is used, as subclasses will also need to define `__slots__` to see the same memory-saving benefits.

In Python, `__slots__` is used to declare a fixed set of attributes for class instances. By doing this, it prevents the creation of an instance dictionary, thus saving memory when you have many instances of a class. `__slots__` is particularly useful for optimizing programs where millions of class instances are created.

Here are three examples demonstrating the use of `__slots__`:

In [8]:

#1. **Basic Usage of `__slots__`**:
 #  A class where instances only have an `x` and `y` attribute, and no other instance attributes can be added.

class Point:
    __slots__ = ('x', 'y')

    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(1, 2)
print(p.x, p.y)  # Output: 1 2
# p.z = 3  # This will raise an AttributeError because z is not in __slots__

1 2


In [ ]:

#2. **Using `__slots__` with Inheritance**:
 #  If a class defines `__slots__`, then its subclasses also need to define `__slots__` to enforce the restriction on instance attributes.

class Shape:
    __slots__ = 'color',

class Circle(Shape):
    __slots__ = 'radius',

    def __init__(self, color, radius):
        self.color = color
        self.radius = radius

c = Circle('blue', 5)
print(c.color, c.radius)  # Output: blue 5
# c.diameter = 10  # This will raise an AttributeError because diameter is not in __slots__


In [ ]:

#3. **Default Values and Descriptors with `__slots__`**:
 #  You can combine `__slots__` with property descriptors or provide default values for the slotted attributes.

class Rectangle:
    __slots__ = ('_width', '_height')

    def __init__(self, width, height):
        self._width = width
        self._height = height

    @property
    def width(self):
        return self._width

    @width.setter
    def width(self, value):
        if value <= 0:
            raise ValueError("Width must be positive")
        self._width = value

    @property
    def height(self):
        return self._height

    @height.setter
    def height(self, value):
        if value <= 0:
            raise ValueError("Height must be positive")
        self._height = value

r = Rectangle(10, 20)
print(r.width, r.height)  # Output: 10 20
r.width = 30
print(r.width)  # Output: 30
# r.width = -5  # This will raise a ValueError because of the check in the setter method


In all of these examples, `__slots__` has been used to declare static data structures to store instance-specific data, which can lead to significant memory savings when dealing with a large number of instances. However, using `__slots__` also comes with some limitations:

- Instances will only be able to have the listed attributes and no others.
- You cannot use `__slots__` with classes meant to be subclassed unless the subclasses also use `__slots__`.
- It's not compatible with some multiple inheritance scenarios.

Therefore, `__slots__` should be used when you are sure about the fixed attributes your instances will have, and you require the memory optimizations it provides.